# Augment the Dataset for 30 EPOCHS(30 Sets of data is enough Augmentations)
- This Speds up computation and training(30 minutes -> 20 Minutes/Epoch)


# Datasets Used:
- 2800 Psuedolabels

Import Dependencies 

In [ ]:
%%capture
# Machine Learning and Data Science Imports
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as backend
import collections
from tensorflow.keras import *
import tensorflow_addons as tfa
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np
import string

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from glob import glob
import random
import math
from tqdm.notebook import tqdm
import os


import matplotlib.pyplot as plt
import cv2

AUTO = tf.data.experimental.AUTOTUNE


# Augmentation and Data Loading Logic

In [ ]:
def compute_area(bbox):
    # Computes Areas of a Bbox
    ones = bbox[:, :2] # (N, 2)
    twos = bbox[:, 2:4] # (N, 2)
    diff = tf.maximum(twos - ones, tf.cast(0.0, dtype = twos.dtype)) # (N, 2)
    return diff[:, 0] * diff[:, 1] # (N,)
def filter_bboxes(box):
    # Filters out Bboxes if their area is < 0
    indices_for_object = tf.where(tf.math.reduce_all([box[:,4] >= 0, 
                                                          box[:, 2] >= box[:, 0], 
                                                          box[:, 3] >= box[:, 1]], axis=0))
    box = tf.cond(tf.shape(indices_for_object)[0] == 0, 
                    lambda: tf.reshape([0., 0., -1., -1., 0.], (-1, 1, box.shape[-1])),
                    lambda: tf.reshape(tf.gather(box, indices_for_object), (-1, 1, box.shape[-1]))) # (?, 1, 5)
    return box
def compute_centers(bbox):
    return 0.5 * (bbox[:, :2] + bbox[:, 2:4])
def compute_iou(bbox, anchors):
    # Computes IOU Between Bbox and Anchors
    # Bbox: tensor(N, min(4)) - As long as the first 4 values are (x1, y1, x2, y2)
    # Anchors: (x1, y1, x2, y2)
    
    # Expand bbox and anchor for broadcasting
    area_bbox = compute_area(bbox) # (N, )
    area_anchor = compute_area(anchors) # (M,)
    
    area_bbox = tf.expand_dims(area_bbox, axis = 0) # (1, N)
    area_anchor = tf.expand_dims(area_anchor, axis = 1) # (M, 1)
    
    
    bbox = tf.expand_dims(bbox, axis = 0) # (1, N, 5)
    anchors = tf.expand_dims(anchors, axis = 1) # (M, 1, 5)
    top_left_bboxes = tf.maximum(bbox[:, :, :2], anchors[:, :, :2]) # (M, N, 2)
    bottom_right_bbox = tf.minimum(bbox[:, :, 2:4], anchors[:, :, 2:4]) # (M, N, 2)
    
    
    
    differences = bottom_right_bbox - top_left_bboxes# (M, N, 2)
    differences = tf.maximum(differences, 0.0)
    
    inter = tf.maximum(differences[:, :, 0] * differences[:, :, 1], 0.0) # (M, N)
    union = area_bbox + area_anchor - inter # (M, N)

    eps = 1e-6
    return (inter + eps) / (union + eps) # (M, N)

In [ ]:
# Helper FN
def filter_normal_bbox(orig_area, bboxes, thresh, min_x_value, max_x_value, min_y_value, max_y_value):
    # orig_area: (N, )
    # Bbox: ((N, 5), (N, C))
    
    # If the area changes too much when clamping the values(Scale, Shift, Rotate, etc,), remove the values
    
    x1 = tf.clip_by_value(bboxes[:, 0], min_x_value, max_x_value) 
    y1 = tf.clip_by_value(bboxes[:, 1], min_y_value, max_y_value)
    x2 = tf.clip_by_value(bboxes[:, 2], min_x_value, max_x_value)
    y2 = tf.clip_by_value(bboxes[:, 3], min_y_value, max_y_value)
    

    x1 = tf.expand_dims(x1, axis = -1)
    y1 = tf.expand_dims(y1, axis = -1)
    x2 = tf.expand_dims(x2, axis = -1)
    y2 = tf.expand_dims(y2, axis = -1)

    bboxes = tf.concat([x1, y1, x2, y2, bboxes[:, 4:]], axis = 1) # (B, 5)
    new_area =  compute_area(bboxes) # (N, )
  
    # Compute the Change in area
    eps = 1e-10
    cng_in_area = (new_area) / (orig_area + eps) # (N, ) - Percentage shrink
    # Threshold
    keep = tf.where(tf.greater_equal(cng_in_area, thresh))
    kept_bboxes = tf.gather(bboxes, keep, axis = 0) # (M, 5)
    return kept_bboxes # (N, 5)

# CROP 

In [ ]:
class RandomCrop(object):
    '''
    Random Square Crop -> Resize(to avoid distortion: distortion is devastating to bounding boxes)
    '''
    def __init__(self, min_height, max_height, thresh = 0.25, p = 0.5):
        self.min_height = min_height
        self.p = p 
        self.max_height = max_height
        self.resizer = Resize(image_size = self.max_height) # Resize After Crops
        self.thresh = thresh # If the bbox decreases in area too much, drop the area
        self.random = tf.data.experimental.RandomDataset(seed = 5)
        self.iterator = iter(self.random)
    def __call__(self, image, bboxes, seeds):
        # Image: Tensor(H, W, C)
        # H and W should be the same.
        # Bboxes: Tensor(N, 5) 
        # classification: Tensor(N, C)
        # Extract values
        #seed1, seed2, seed3, seed4 = seeds
        seed1 = next(self.iterator)
        seed2 = next(self.iterator)
        seed3 = next(self.iterator)
        seed4 = next(self.iterator)
        if len(bboxes.shape) > 2:
          bboxes = tf.squeeze(bboxes, axis = 1)
        
        seed = next(self.iterator)
        seed = seed % int(1e5)
        seed = seed / int(1e5)
        if seed > self.p:
            return tf.cast(image, dtype = tf.float32), tf.cast(bboxes, dtype = tf.float32)
        # compute Original Area
        orig_area = compute_area(bboxes) # (N, )
        # ------------------
        # Find a Random Crop:


        width = seed1 % (self.max_height - self.min_height) + self.min_height
        height = seed2 % (self.max_height - self.min_height) + self.min_height # My version of random sampling.
        # Clip the Width and Height


        starting_x = seed3 % (self.max_height - width)
        starting_y = seed4 % (self.max_height - height)
        
       
        starting_x = tf.cast(starting_x, tf.int32)
        starting_y = tf.cast(starting_y, tf.int32)
        width = tf.cast(width, tf.int32)
        height = tf.cast(height, tf.int32)
        # Shift Bbox Coords by the starting_x and starting_y, to create relative x, and y values.
        
        x1 = bboxes[:, 0] - tf.cast(starting_x, bboxes.dtype)
        y1 = bboxes[:, 1] - tf.cast(starting_y, bboxes.dtype)
        x2 = bboxes[:, 2] - tf.cast(starting_x, bboxes.dtype)
        y2 = bboxes[:, 3] - tf.cast(starting_y, bboxes.dtype)
        obj = bboxes[:, 4:]

        x1 = tf.expand_dims(x1, axis = -1)
        x2 = tf.expand_dims(x2, axis = -1)
        y1 = tf.expand_dims(y1, axis = -1)
        y2 = tf.expand_dims(y2, axis = -1)

        new_bboxes = tf.concat([x1, y1, x2, y2, obj], axis = 1) # (N, 5)
        
        # Crop the Images
        new_image = image[starting_y: starting_y + height, starting_x: starting_x + width, :]
        
        kept_bboxes = tf.reshape(new_bboxes, (-1, 5))
        kept_bboxes = filter_normal_bbox(orig_area, kept_bboxes, self.thresh, tf.cast(tf.constant(0.0), kept_bboxes.dtype), tf.cast(width, kept_bboxes.dtype), tf.cast(tf.constant(0.0), kept_bboxes.dtype), tf.cast(height, kept_bboxes.dtype))
        kept_bboxes = tf.reshape(new_bboxes, (-1, 5))
        
        # Resize the Bounding Boxes and Image
        new_image, kept_bboxes = self.resizer(new_image, kept_bboxes, height, width)
        
        return tf.cast(new_image, dtype = tf.float32), tf.cast(kept_bboxes, dtype = tf.float32)

# Transpose

In [ ]:
def transpose(bboxes):
  x1 = bboxes[:, 0]
  y1 = bboxes[:, 1]
  x2 = bboxes[:, 2]
  y2 = bboxes[:, 3]
  obj = bboxes[:, 4:]

  x1, y1 = (y1, x1)
  x2, y2 = (y2, x2)

  x1 = tf.expand_dims(x1, axis = 1)
  x2 = tf.expand_dims(x2, axis = 1)
  y1 = tf.expand_dims(y1, axis = 1)
  y2 = tf.expand_dims(y2, axis = 1) 

  bboxes = tf.concat([x1, y1, x2, y2, obj], axis = 1)
  return bboxes
class Transpose(object):
  def __init__(self, p = 0.5):
    self.p = p
    self.random = tf.data.experimental.RandomDataset(seed = 3)
    self.iterator = iter(self.random)
  def __call__(self, images, bboxes, seeds):
    seed = next(self.iterator)
    seed = seed % int(1e5)
    seed = seed / int(1e5)
    if seed > self.p:
      return images, bboxes
    images = tf.squeeze(tf.image.transpose(tf.expand_dims(images, axis = 0)), axis = 0)
    x1 = bboxes[:, 0]
    y1 = bboxes[:, 1]
    x2 = bboxes[:, 2]
    y2 = bboxes[:, 3]
    obj = bboxes[:, 4:]

    x1, y1 = (y1, x1)
    x2, y2 = (y2, x2)

    x1 = tf.expand_dims(x1, axis = 1)
    x2 = tf.expand_dims(x2, axis = 1)
    y1 = tf.expand_dims(y1, axis = 1)
    y2 = tf.expand_dims(y2, axis = 1) 

    bboxes = tf.concat([x1, y1, x2, y2, obj], axis = 1)
    return images, bboxes 

# Rotate 90

In [ ]:
class Rotate90(object):
    def __init__(self, image_size, p = 0.5, thresh = 0.25):
        self.image_size = image_size
        self.p = p
        self.thresh = thresh
        self.random = tf.data.experimental.RandomDataset()
        self.iterator = iter(self.random)
    
    
    def __call__(self, images, bboxes, seed):
        random_seed = next(self.iterator)
        random_seed = random_seed % int(1e5)
        random_seed = random_seed / int(1e5)
        if random_seed > self.p:
            return images, bboxes
        #seed1, seed2, seed3, seed4 = seed
        seed1 = next(self.iterator)
        seed2 = next(self.iterator)
        seed3 = next(self.iterator)
        seed4 = next(self.iterator)
        
        seed1 = (seed1 % 3) + 1
        k = seed1
    
        images = tf.image.rot90(images, k = tf.cast(k, tf.int32))
        orig_area = compute_area(bboxes)

        if k == 1:
          bboxes = horizontal_flip(images, bboxes)
          bboxes = transpose(bboxes)
        elif k == 2:
          bboxes = horizontal_flip(images, bboxes)
          bboxes = vertical_flip(images, bboxes)
        else:
          bboxes = vertical_flip(images, bboxes)
          bboxes = transpose(bboxes)
      
        bboxes = filter_normal_bbox(orig_area, bboxes, self.thresh, tf.cast(tf.constant(0.), bboxes.dtype), tf.cast(tf.constant(self.image_size), bboxes.dtype), tf.cast(tf.constant(0.0), bboxes.dtype), tf.cast(tf.constant(self.image_size),bboxes.dtype))
        bboxes = tf.reshape(bboxes, (-1, 5))
        return images, bboxes

# Sharpen

In [ ]:
class Sharpen(object):
    def __init__(self, min_alpha = 0.2, max_alpha = 0.5, min_sharpness = 0.5, max_sharpness = 1.0, p = 0.5):
        self.min_alpha = min_alpha
        self.max_alpha = max_alpha
        self.p = p
        self.min_sharpness = min_sharpness
        self.max_sharpness = max_sharpness

        self.random = tf.data.experimental.RandomDataset()
        self.iterator = iter(self.random)
    def __call__(self, images, bboxes, seeds):
        random_seed = next(self.iterator)
        random_seed = random_seed % int(1e5)
        random_seed = random_seed / int(1e5)

        if random_seed > self.p:
            return images, bboxes
        #seed1, seed2, seed3, seed4 = seeds
        seed1 = next(self.iterator)
        seed2 = next(self.iterator)
        seed3 = next(self.iterator)
        seed4 = next(self.iterator)
        
        seed1 = seed1 % int(1e5) # Number of Buckets
        seed1 = seed1 / int(1e5) # convert to uniform number

        seed2 = seed2 % int(1e5)
        seed2 = seed2 / int(1e5)
        alpha = seed1
        sharpness = seed2
        
        sharpened = tfa.image.sharpness(images, factor = tf.cast(sharpness, images.dtype)) 
        return images * tf.cast(1 - alpha, images.dtype) + tf.cast(alpha, sharpened.dtype) * sharpened, bboxes


# Mosaic 

In [ ]:
def crop_range(bboxes, h_min_bound, h_max_bound, w_min_bound, w_max_bound, thresh = 0.05):
  # Clips Bounding Boxes and Turns invalid bounding boxes to 0(obj = 0)
  
  orig_area = compute_area(bboxes)

  x1 = bboxes[:, 0]
  y1 = bboxes[:, 1]
  x2 = bboxes[:, 2]
  y2 = bboxes[:, 3]
  obj = bboxes[:, 4]

  x1 = tf.clip_by_value(x1, w_min_bound, w_max_bound)
  y1 = tf.clip_by_value(y1, h_min_bound, h_max_bound)
  x2 = tf.clip_by_value(x2, w_min_bound, w_max_bound)
  y2 = tf.clip_by_value(y2, h_min_bound, h_max_bound)

  x1 = tf.expand_dims(x1, axis = 1)
  y1 = tf.expand_dims(y1, axis = 1)
  x2 = tf.expand_dims(x2, axis = 1)
  y2 = tf.expand_dims(y2, axis = 1)

  bboxes = tf.concat([x1, y1, x2, y2, obj[:, None]], axis = 1)
  # Compute the Area
  area = compute_area(bboxes) # (N, )
  # Filter out the invalid bounding boxes
  cng_in_area = area / orig_area
  keep = tf.cast(tf.greater(cng_in_area, thresh), area.dtype)
  obj = obj * keep

  bboxes = tf.concat([x1, y1, x2, y2, obj[:, None]], axis = 1)
  
  return bboxes

def shift_bounding_boxes(bboxes, h_shift, w_shift):
  x1 = tf.expand_dims(bboxes[:, 0] - w_shift, axis = 1)
  y1 = tf.expand_dims(bboxes[:, 1] - h_shift, axis = 1)
  x2 = tf.expand_dims(bboxes[:, 2] - w_shift, axis = 1)
  y2 = bboxes[:, 3] - h_shift 
  obj = bboxes[:, 4:]

  boxes = tf.concat([x1, y1, x2, y2, obj], axis = 1)
  return boxes

class CustomMosaic(object):
    # Corners Mosaic.
    def __init__(self, p = 0.5, thresh = 0.05, border = 40):
        self.p = p
        self.border = border
        self.thresh = thresh
        # So many random seeds that a generator is needed
        self.generator = tf.data.experimental.RandomDataset()
        self.iterator = iter(self.generator)
    
    def __call__(self, images, bboxes, seeds):
        B, H, W, C = images.shape
        shift = H // 2
        # Reshape the Bboxes
        bboxes = tf.reshape(bboxes, (B, -1, 5))
        B, N, _ = bboxes.shape
        IMAGES = tf.ones((0, H, W, C))
        BBOXES = tf.ones((0, N * 4, 5))
        # bboxes: (B, N, 5) 
        for b in range(B):
          value = next(self.iterator) % int(1e5)
          value = value / int(1e5)
          #tf.print('---------ITERATION--------------')
          if value > self.p:
            # Stack bboxes 4x
            #tf.print("---------Skip-----------")
            image1 = images[b]
            bboxes1 = bboxes[b]
            # Bounding Boxes set to 0
            bbox0 = tf.zeros_like(bboxes1, dtype = bboxes1.dtype) 
            #tf.print(bboxes1.shape, bbox0.shape)
            boxes = tf.concat([bboxes1, bbox0, bbox0, bbox0], axis = 0)
            
            #tf.print(boxes.shape, BBOXES.shape)
            #tf.print(image1.shape, IMAGES.shape)
            IMAGES = tf.concat([IMAGES, tf.expand_dims(image1, axis = 0)], axis = 0)
            BBOXES = tf.concat([BBOXES, tf.expand_dims(boxes, axis = 0)], axis = 0)
            
          else:
            #tf.print('----------MOSAIC---------')

            # select 4 Corners of the image
            x1 = next(self.iterator) % (W - self.border) + self.border
            y1 = next(self.iterator) % (H - self.border) + self.border 
          

            
            # Select the Corners from four random images
            image1 = images[b] # (H, W, 3)
            bboxes1 = bboxes[b] # (N, 5)
            # Select More images(3 extra)
            idx2 = next(self.iterator) % B
            idx3 = next(self.iterator) % B
            idx4 = next(self.iterator) % B 
            image2 = images[idx2]
            bboxes2 = bboxes[idx2]

            image3 = images[idx3]
            bboxes3= bboxes[idx3]

            image4 = images[idx4]
            bboxes4 = bboxes[idx4]
            
            # Grab out the corners
            corner1 = image1[y1:, x1:] # Bottom Right of Image
            corner2 = image2[:y1, x1:] # Top Right of Image 
            corner3 = image3[y1:, :x1] # Bottom Left of Image
            corner4 = image4[:y1, :x1] # Top Left of Image

            # Crop and Filter bounding boxes
            bboxes1 = crop_range(bboxes1, tf.cast(y1, bboxes1.dtype), tf.cast(H, bboxes1.dtype), tf.cast(x1, bboxes1.dtype), tf.cast(W, bboxes1.dtype), thresh = self.thresh)
            bboxes2 = crop_range(bboxes2, tf.constant(0.0, bboxes2.dtype), tf.cast(y1, bboxes2.dtype), tf.cast(x1, bboxes2.dtype), tf.cast(W, bboxes2.dtype), thresh = self.thresh)
            bboxes3 = crop_range(bboxes3, tf.cast(y1, bboxes3.dtype), tf.cast(H, bboxes.dtype), tf.constant(0.0, bboxes3.dtype), tf.cast(x1, bboxes.dtype), thresh = self.thresh)
            bboxes4 = crop_range(bboxes4, tf.constant(0.0, bboxes4.dtype), tf.cast(y1, bboxes4.dtype), tf.constant(0.0, bboxes4.dtype), tf.cast(x1, bboxes4.dtype), thresh = self.thresh)
            
            # Concatenate the Corners
            top_half = tf.concat([corner2, corner1], axis = 0)
            bottom_half = tf.concat([corner4, corner3], axis = 0)
            full_image = tf.concat([bottom_half, top_half], axis = 1)
            # concatenate the bounding boxes.
            #tf.print(bboxes1.shape, bboxes2.shape, bboxes3.shape, bboxes4.shape)
            
            boxes = tf.concat([bboxes1, bboxes2, bboxes3, bboxes4], axis = 0)
            

            #tf.print(BBOXES.shape, boxes.shape)
            IMAGES = tf.concat([IMAGES, tf.expand_dims(full_image, axis = 0)], axis = 0)
            BBOXES = tf.concat([BBOXES, tf.expand_dims(boxes, axis = 0)], axis = 0)
            #tf.print(BBOXES.shape)
        return IMAGES, BBOXES

# Mixup

In [ ]:
class Mixup(object):
  # object detection Mixup
  # Just Average 2 Images, and concatenate the bounding boxes.

  def __init__(self, p = 0.5):
    super().__init__()
    self.p = p
    self.generator = tf.data.experimental.RandomDataset()
    self.iterator = iter(self.generator)
  def __call__(self, images, bboxes, seeds):
   
    # Images: Tensor(B, H, W, 3)
    # Bboxes: Tensor(B, 5) -> Batched Image Bbox Pairs.
    
    B, H, W, C = images.shape
    _, N, num_channels = bboxes.shape
    IMAGES = tf.ones((0, H, W, C), images.dtype)
    BBOXES = tf.ones((0, N * 2, num_channels), images.dtype)

    for b in range(B):
      base_image = images[b] # (H, W, 3)
      base_bbox = bboxes[b]  # (N, 5)
      seed = next(self.iterator) % int(1e5)
      seed = seed / int(1e5)
      if seed > self.p:
        new_images = base_image
        #tf.print(base_bbox.shape, black_bbox.shape)
        new_bboxes = tf.concat([base_bbox, tf.zeros_like(base_bbox, dtype = base_bbox.dtype)], 0)

        IMAGES = tf.concat([IMAGES, tf.expand_dims(new_images, axis = 0)], axis = 0) 
        BBOXES = tf.concat([BBOXES, tf.expand_dims(new_bboxes, axis = 0)], axis = 0)

      else:
        #tf.print('--------Not skip----------')
        # Select another image
        selected_index = next(self.iterator) % B
        selected_image = images[selected_index] # (H, W, 3)
        selected_bbox = bboxes[selected_index] # (N, 5) 
        #tf.print(images.shape, bboxes.shape)
        #tf.print(selected_image.shape, selected_bbox.shape)
        # Mixup
        #tf.print(base_bbox.shape, selected_bbox.shape)

        new_image = (base_image + selected_image) / 2
        new_bbox = tf.concat([base_bbox, selected_bbox], axis = 0)
        #tf.print(IMAGES.shape, new_image.shape)
        #tf.print(BBOXES.shape, new_bbox.shape)
        IMAGES = tf.concat([IMAGES, tf.expand_dims(new_image, axis = 0)], axis = 0)
        BBOXES = tf.concat([BBOXES, tf.expand_dims(new_bbox, axis = 0)], axis = 0)

    IMAGES = tf.reshape(IMAGES, (B, H, W, C))
    BBOXES = tf.reshape(BBOXES, (B, N * 2, num_channels))

    return IMAGES, BBOXES   

In [ ]:
def cut_one_hole(image, min_x, min_y, mask_size):
  # KEY ASSUMPTION: Batch size must be 1.
  IMAGE_SIZE = tf.constant(DataModule.IMG_SHAPE[0])

  B, H, W, _ = image.shape

  image = tf.reshape(image, (B, IMAGE_SIZE, IMAGE_SIZE, -1))
  B, H, W, C = image.shape
  
  max_x = min_x + mask_size
  max_y = min_y + mask_size
  
  # Zero Tensor
  zero_values = tf.zeros((B, mask_size, mask_size, C), image.dtype)

  # Compute the Left Array of Ones
  left_tensor = tf.ones((B, H, min_x, C), image.dtype)
  right_tensor = tf.ones((B, H, W - max_x, C), image.dtype) 

  # Top Part
  top_part = tf.ones((B, min_y, mask_size, C), image.dtype)
  bottom_part = tf.ones((B, H - max_y, mask_size, C), image.dtype)

  middle_portion = tf.concat([top_part, zero_values, bottom_part], axis = 1) # (B, H, mask_size, C)
  mask = tf.concat([left_tensor, middle_portion, right_tensor], axis = 2)

  return tf.cast(mask * image, dtype = tf.float32)
class CustomCutout(object):
    # Simple Random Cutout, as it is done with Albumentations
    # SHOULD BE APPLIED AFTER MIXUP and MOSAIC, In a BATCHED manner
    def __init__(self, mask_size, num_masks, p = 0.5):
        self.mask_size = mask_size
        self.num_masks = num_masks 
        self.p = p 
        self.seed = 42
        self.generator = tf.data.experimental.RandomDataset()
        self.iterator = iter(self.generator)
    def __call__(self, images, bboxes, seeds):
        B, H, W, C = images.shape
        B, N, num_classes = bboxes.shape
        IMAGES = tf.ones((0, H, W, C), images.dtype)
        BBOXES = tf.ones((0, N, num_classes), bboxes.dtype)
        # Nothing to do with bboxes
        for b in range(B):
          seed = next(self.iterator)
          seed = seed % int(1e5)
          seed = seed / int(1e5)
          if seed > self.p:
            new_images = tf.cast(tf.expand_dims(images[b], 0), dtype = images.dtype)
            new_bboxes = tf.cast(tf.expand_dims(bboxes[b], 0), dtype = bboxes.dtype)
            IMAGES = tf.cast(tf.concat([IMAGES, new_images], axis = 0), dtype = images.dtype)
            BBOXES = tf.cast(tf.concat([BBOXES, new_bboxes], axis = 0), dtype = bboxes.dtype)
          else:
            new_images = tf.cast(tf.expand_dims(images[b], 0), dtype = images.dtype)
            new_bboxes = tf.cast(tf.expand_dims(bboxes[b], 0), dtype = bboxes.dtype)
            for i in range(self.num_masks):
              # cutout a box
              # Compute a Random Place
              x = next(self.iterator) % (W - self.mask_size)
              y = next(self.iterator) % (H - self.mask_size)

              new_images = tf.cast(cut_one_hole(new_images, x, y, self.mask_size), dtype = images.dtype)
            IMAGES = tf.cast(tf.concat([IMAGES, new_images], axis = 0), dtype = images.dtype)
            BBOXES = tf.cast(tf.concat([BBOXES, new_bboxes], axis = 0), dtype = bboxes.dtype)
        return IMAGES, BBOXES

In [ ]:
def horizontal_flip(img, bboxes):
  H, W, C = img.shape
  IMG_CENTER = W // 2 if W else DataModule.IMG_SHAPE[1] // 2
  
  x1 = tf.expand_dims(bboxes[:, 0], 1)
  y1 = tf.expand_dims(bboxes[:, 1], 1)
  x2 = tf.expand_dims(bboxes[:, 2], 1)
  y2 = tf.expand_dims(bboxes[:, 3], 1)
  obj = bboxes[:, 4:]
  
  
  x1 = x1 + 2 * (IMG_CENTER - x1)
  x2 = x2 + 2 * (IMG_CENTER - x2) 

  box_w = tf.abs(x2 - x1)

  x1 = x1 - box_w
  x2 = x2 + box_w

  
  bboxes = tf.concat([x1, y1, x2, y2, obj], axis = 1)
  return bboxes
def vertical_flip(img, bboxes):
  H, W, C = img.shape
  img_center = H // 2 if H else DataModule.IMG_SHAPE[0] // 2
  
  x1 = tf.expand_dims(bboxes[:, 0], 1)
  y1 = tf.expand_dims(bboxes[:, 1], 1)
  x2 = tf.expand_dims(bboxes[:, 2], 1)
  y2 = tf.expand_dims(bboxes[:, 3], 1)
  obj = bboxes[:, 4:]

  y1 = y1 + 2 * (img_center - y1)
  y2 = y2 + 2 * (img_center - y2)

  box_h = tf.abs(y2 - y1)

  y1 = y1 - box_h
  y2 = y2 + box_h

  bboxes = tf.concat([x1, y1, x2, y2, obj], axis = 1)
  bboxes = tf.reshape(bboxes, (-1, 5))

  return bboxes
class Resize(object):
    """Resize the image in accordance to `image_letter_box` function in darknet 
    
    The aspect ratio is maintained. The longer side is resized to the input 
    size of the network, while the remaining space on the shorter side is filled 
    with black color. **This should be the last transform**
    
    
    Parameters
    ----------
    inp_dim : tuple(int)
        tuple containing the size to which the image will be resized.
        
    Returns
    -------
    
    numpy.ndaaray
        Sheared image in the numpy format of shape `HxWxC`
    
    numpy.ndarray
        Resized bounding box co-ordinates of the format `n x 4` where n is 
        number of bounding boxes and 4 represents `x1,y1,x2,y2` of the box
        
    """
    
    def __init__(self, image_size = 1024):
        self.inp_dim = tf.constant(image_size)
        
    def __call__(self, img, bboxes, height, width):
        
        w,h = width, height
        img = tf.image.resize(img, (self.inp_dim, self.inp_dim), antialias = True)
        
        if h is None:
          return img, bboxes
       

        w_scale = tf.cast(self.inp_dim / w, bboxes.dtype)
        h_scale = tf.cast(self.inp_dim / h, bboxes.dtype)

        x1 = bboxes[:, 0] * w_scale
        y1 = bboxes[:, 1] * h_scale
        x2 = bboxes[:, 2] * w_scale
        y2 = bboxes[:, 3] * h_scale
        obj = bboxes[:, 4:]

        x1 = tf.expand_dims(x1, 1)
        y1 = tf.expand_dims(y1, 1)
        x2 = tf.expand_dims(x2, 1)
        y2 = tf.expand_dims(y2, 1)

        bboxes = tf.concat([
        x1, y1, x2, y2, obj
        ], axis = 1)
        return img, bboxes 
    
class RandomHorizontalFlip(object):

    """Randomly horizontally flips the Image with the probability *p*
    Parameters
    ----------
    p: float
        The probability with which the image is flipped
    Returns
    -------
    numpy.ndaaray
        Flipped image in the numpy format of shape `HxWxC`
    numpy.ndarray
        Tranformed bounding box co-ordinates of the format `n x 4` where n is
        number of bounding boxes and 4 represents `x1,y1,x2,y2` of the box
    """

    def __init__(self, p=0.5):
        self.p = p
        self.random = tf.data.experimental.RandomDataset()
        self.iterator = iter(self.random)
    def __call__(self, img, bboxes, seeds):
        seed = next(self.iterator)
        seed = seed % int(1e5)
        seed = seed / int(1e5)
        if seed > self.p:
          return img, bboxes
        H, W, C = img.shape
        IMG_CENTER = W // 2 if W else DataModule.IMG_SHAPE[1] // 2
        img = img[:, ::-1]
        
        x1 = tf.expand_dims(bboxes[:, 0], 1)
        y1 = tf.expand_dims(bboxes[:, 1], 1)
        x2 = tf.expand_dims(bboxes[:, 2], 1)
        y2 = tf.expand_dims(bboxes[:, 3], 1)
        obj = bboxes[:, 4:]
        
        
        x1 = x1 + 2 * (IMG_CENTER - x1)
        x2 = x2 + 2 * (IMG_CENTER - x2) 

        box_w = tf.abs(x2 - x1)

        x1 = x1 - box_w
        x2 = x2 + box_w

        
        bboxes = tf.concat([x1, y1, x2, y2, obj], axis = 1)

        return img, bboxes
class RandomVerticalFlip(object):
    def __init__(self, p=0.5):
        self.p = p
        self.random = tf.data.experimental.RandomDataset()
        self.iterator = iter(self.random)

    def __call__(self, img, bboxes, seeds):
        seed = next(self.iterator)
        seed = seed % int(1e5)
        seed = seed / int(1e5)
        if seed > self.p:
          return img, bboxes

        H, W, C = img.shape
        img_center = H // 2 if H else DataModule.IMG_SHAPE[0] // 2
        
        img = img[::-1, :, :]
        x1 = tf.expand_dims(bboxes[:, 0], 1)
        y1 = tf.expand_dims(bboxes[:, 1], 1)
        x2 = tf.expand_dims(bboxes[:, 2], 1)
        y2 = tf.expand_dims(bboxes[:, 3], 1)
        obj = bboxes[:, 4:]

        y1 = y1 + 2 * (img_center - y1)
        y2 = y2 + 2 * (img_center - y2)

        box_h = tf.abs(y2 - y1)

        y1 = y1 - box_h
        y2 = y2 + box_h

        bboxes = tf.concat([x1, y1, x2, y2, obj], axis = 1)
        bboxes = tf.reshape(bboxes, (-1, 5))

        return img, bboxes

In [ ]:
class RandomResizedCrop(object):
  def __init__(self, min_height, max_height, thresh = 0.25, crop_prob = 0.75, image_size = 1024):
    self.crop = RandomCrop(min_height, max_height, thresh = thresh, p = crop_prob)
  def __call__(self, images, bboxes, seeds):
    #print(images.shape, bboxes.shape)
    images, bboxes = self.crop(images, bboxes, seeds)
    return images, bboxes

# Dataset Encoding

In [ ]:
# BBox Alignment Code
def datasets_encode(x, y, overlap_threshold = 0.5, ignore_threshold = 0.4):
    # x (Batch Size, H, W, 3)
    # y (Batch Size, N, 5)  x1, y1, x2, y2, class

    # resize image
    WIDTH, HEIGHT, _ = DataModule.IMG_SHAPE
    if x.shape == DataModule.IMG_SHAPE:
        x_scale, y_scale = DataModule.IMG_SHAPE[0] / x.shape[0], DataModule.IMG_SHAPE[1] / x.shape[1]
        y = y * [x_scale, y_scale, x_scale, y_scale, 1]
        x = tf.image.resize(x, [DataModule.IMG_SHAPE[0], DataModule.IMG_SHAPE[1]])
    

    assignment_x = tf.zeros((DataModule.ANCHORS.shape[0]), tf.float32)
    assignment_y = tf.zeros((DataModule.ANCHORS.shape[0]), tf.float32)
    assignment_w = tf.zeros((DataModule.ANCHORS.shape[0]), tf.float32)
    assignment_h = tf.zeros((DataModule.ANCHORS.shape[0]), tf.float32)
    assignment_classes_idx = tf.zeros((DataModule.ANCHORS.shape[0]), tf.float32) + DataModule.NUM_CLASSES
    assignment_is_obj = tf.zeros((DataModule.ANCHORS.shape[0]), tf.float32)


    # filter correct boxes (area > 0)
    priors = DataModule.ANCHORS # (1, 196416, 4)
    width = tf.expand_dims(priors[:, 2] - priors[:, 0], axis = 1)
    height = tf.expand_dims(priors[:, 3] - priors[:, 1], axis = 1)
    
    anchor_wh = tf.concat([width, height], axis = 1) # (196416, 2)
    
    box = tf.squeeze(filter_bboxes(y), axis = 1)
   
    iou = compute_iou(priors, box)

    iou_max = tf.math.reduce_max(iou, axis=0) # (196416)
    iou_max_idxs = tf.math.argmax(iou, axis=0) # (196416)
    
    # ignore box
    ignore_mask = tf.math.logical_and(iou_max > ignore_threshold, iou_max < overlap_threshold) # (196416)
    assignment_is_obj = tf.where(ignore_mask, -1., assignment_is_obj)

    # object box
    assign_mask = iou_max > overlap_threshold # (196416)
    #print(tf.where(assign_mask))
    box_best = tf.gather(box, iou_max_idxs, axis=0) # (?, 5) + (196416) => (196416, 5)_
    assignment_is_obj = tf.where(assign_mask, 1., assignment_is_obj)
    assignment_classes_idx = tf.where(assign_mask, box_best[:, 4], assignment_classes_idx)



    box_center = compute_centers(box_best)

    box_wh = box_best[:,2:4] - box_best[:,:2]  #  (196416, 2)
    assigned_xy = (box_center - compute_centers(priors)) #  (196416, 2)
    assigned_wh = tf.math.log(box_wh / anchor_wh) #  (196416, 2)
    assignment_x = tf.where(assign_mask, assigned_xy[:, 0], assignment_x)
    assignment_y = tf.where(assign_mask, assigned_xy[:, 1], assignment_y)
    assignment_w = tf.where(assign_mask, assigned_wh[:, 0], assignment_w)
    assignment_h = tf.where(assign_mask, assigned_wh[:, 1], assignment_h)


    regression_list = tf.stack([
        assignment_x, assignment_y, 
        assignment_w, assignment_h, 
        assignment_is_obj], axis=-1) # (N, 5)  
    

    classification_list = tf.one_hot(tf.cast(assignment_classes_idx, tf.int32), DataModule.NUM_CLASSES + 1) 
        
    
    return x, (regression_list, classification_list)
def batched_dataset_encode(images, bboxes, overlap_threshold = 0.5, ignore_threshold = 0.4):
  # Images: tensor(B, H, W, 3)
  # Bboxes: (Tensor(B, N,5), Tensor(B, N, C))
  B, H, W, C = images.shape
  _, _, num_channels = bboxes.shape
  num_anchors = len(DataModule.ANCHORS)
  num_classes = DataModule.NUM_CLASSES if DataModule.NUM_CLASSES != 1 else 2 
  IMAGES = tf.ones((0, H, W, C), images.dtype)
  BBOXES = tf.ones((0, num_anchors, num_channels), images.dtype)
  CLASSIFICATION = tf.ones((0, num_anchors, num_classes), images.dtype)
  for b in range(B):
    image = images[b] # (H, W, 3)
    bbox = bboxes[b] # (N, 5)
    
    # Drop Padded Values: Where bboxes[:, -1] == 0.0
    keep = tf.equal(bbox[:, -1], 1.0)
  
    bbox = tf.boolean_mask(bbox, keep, axis = 0)

    image, (bbox, Class) = datasets_encode(image, bbox, overlap_threshold = overlap_threshold, ignore_threshold = ignore_threshold)
    
    IMAGES = tf.concat([IMAGES, tf.expand_dims(image, axis = 0)], axis = 0)
    BBOXES = tf.concat([BBOXES, tf.expand_dims(bbox, axis = 0)], axis = 0)
    CLASSIFICATION = tf.concat([CLASSIFICATION, tf.expand_dims(Class, axis = 0)], axis = 0)
  return IMAGES, (BBOXES, CLASSIFICATION)


In [ ]:
class Mosaic_Plus_Mixup(object):
    def __init__(self, p = 0.33):
        self.p = p
        self.mosaic = CustomMosaic(p = 1.)
        self.mixup = Mixup(p = 1.)
    def __call__(self, images, bboxes, seeds):
        seed = seeds[0]
        seed = seed % 1000
        seed = seed / 1000
    
        if seed < self.p:
            zero_bboxes = tf.zeros_like(bboxes, dtype = bboxes.dtype)
            bboxes = tf.concat([bboxes, zero_bboxes, zero_bboxes, zero_bboxes], axis = 1) 
        elif seed < self.p * 2:
            # MOSAIC
            images, bboxes = self.mosaic(images, bboxes, seeds) 
        else:
            # MIXUP 
            images, bboxes = self.mixup(images, bboxes, seeds)
            zero_bboxes = tf.zeros_like(bboxes, dtype = bboxes.dtype)
            bboxes = tf.concat([bboxes, zero_bboxes], axis = 1)
            
        return images, bboxes

In [ ]:
class Augments:
  IMAGE_SIZE = 1024
  THRESH = 0.25 # Doesn't Matter how large the bounding box is: Just can't be completely outside of the image.
  BORDER = 100
  # ------Augmentation----------
  resize = RandomResizedCrop(800, IMAGE_SIZE, thresh = THRESH, crop_prob = 0.75, image_size = IMAGE_SIZE)
  hori_flip = RandomHorizontalFlip(p = 0.5)
  vert_flip = RandomVerticalFlip(p = 0.5)
  rotate90 = Rotate90(IMAGE_SIZE, p = 0.5, thresh = THRESH)
  cutout = CustomCutout(64, 8, p = 0.5)
  sharpen = Sharpen(p = 0.5)
  mosaic_mixup = Mosaic_Plus_Mixup(p = 0.33) # P = 0.33, Mixup = 0.33, Mosaic = 0.33
  mosaic = CustomMosaic(p = 0.5)
  mixup = Mixup(p = 0.5)

  transpose = Transpose(p = 0.5)
  
  all_augs = [
      hori_flip,
      vert_flip,
      rotate90,
      transpose,
      #sharpen,
      
  ]
  batched_augs = [
      #mosaic,
      #mixup,
      mosaic_mixup,
      cutout
  ]

  MAX_BBOX = 200 # Assume Max # of Bboxes, to pad to.(You can tune this to the dataset)


In [ ]:
def IMAGE_LOAD(example):
    feature_dict = {
        'image': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=''),
        'bboxes': tf.io.VarLenFeature(dtype = tf.int64)
    }
    features = tf.io.parse_single_example(example, features=feature_dict)
    
    image = features['image']
    bboxes = features['bboxes'] 
    
    # Load image
    image = tf.io.decode_jpeg(image, channels = 3)
    #Cast to Desired Dtype
    image = tf.cast(image, tf.float32) / 255.0
    # Reshape the image 
    image = tf.reshape(image,  DataModule.IMG_SHAPE)
    # Load the Bounding Boxes
    bboxes = tf.cast(tf.sparse.to_dense(bboxes), tf.float32)
    bboxes = tf.reshape(bboxes, (-1, 4))
    # all Bounding boxes there are class 1(Obviously.)
    values = tf.identity(bboxes)[:, 0] * 0.0 + 1.0
    bboxes = tf.concat([bboxes, tf.expand_dims(values, axis = 1)],axis = 1) 
    
    return image, bboxes


In [ ]:
def thresh_by_zero(bboxes):
  img_shape = Augments.IMAGE_SIZE
  # Bboxes: Tensor(B, 5)
  x1 = bboxes[:, 0]
  y1 = bboxes[:, 1]
  x2 = bboxes[:, 2]
  y2 = bboxes[:, 3]
  obj = bboxes[:, 4:]

  # Threshold the Values
  x1 = tf.expand_dims(tf.clip_by_value(x1, 0.0, img_shape - 1), axis = -1)
  y1 = tf.expand_dims(tf.clip_by_value(y1, 0.0, img_shape - 1), axis = -1)
  x2 = tf.expand_dims(tf.clip_by_value(x2, 0.0, img_shape - 1), axis = -1)
  y2 = tf.expand_dims(tf.clip_by_value(y2, 0.0, img_shape - 1), axis = -1)
  # Compute Area and Threshold
  bboxes = tf.concat([x1, y1, x2, y2, bboxes[:, 4:]], axis = 1) # (B, 5)
  new_area =  compute_area(bboxes) # (N, )

  # Threshold
  keep = tf.where(tf.greater(new_area, 0.0))
  kept_bboxes = tf.gather(bboxes, keep, axis = 0) # (M, 5)
  return kept_bboxes # (N, 5)

In [ ]:
def load_tfrecs(data_dir):
    TFRECS = glob(f"{data_dir}/*")
    return TFRECS
def load_folds(fold_tfrecs, *args):
    # Args is the list of all other TFRECS, added to train(pseudo + external)
    FOLDS = []
    for i in range(len(fold_tfrecs)):
        TRAIN = []
        TEST = []
        for tfrec in fold_tfrecs:
            ending = ''
            for idx in range(len(tfrec)-1, -1, -1):
                if tfrec[idx] == '/':
                    ending = tfrec[idx + 1:]
                    break
            if str(i) in ending:
                TEST += [tfrec]
            else:
                TRAIN += [tfrec]
        for arg in args:
            TRAIN += arg
        FOLDS += [(TRAIN, TEST)]
    return FOLDS
        
class DataModule:
    # -------------Load Dataset----------------
    data_dir = 'gwd2021'
    base_dir = f'../input/{data_dir}'
    SAVE_PATH = './'
    # Load FOLD idx
    FOLD_TFRECS = load_tfrecs(base_dir)
    # Load external Data
    FOLDS = load_folds(FOLD_TFRECS)
    # -----------OTHER Data Params-------------------
    IMG_SHAPE = (1024, 1024, 3) # Massive Fricken Images - But EfficientDet + TPUs can handle it.
    # Generate Anchors, a Constant.
    # num Classes = 1. Wheat or No Wheat
    NUM_CLASSES = 1
    # Max Number of BBoxes == 516, but to be safe, 1024 cap.
    MAX_BBOXES = 1024
    # Total Number
    TRAIN_NUMBER = sum(1 for _ in tf.data.TFRecordDataset(FOLD_TFRECS))

In [ ]:
# LOAD IN THE DATASET
FILE_PATH = '../input/gwd-069/submission.csv'
df = pd.read_csv(FILE_PATH)

In [ ]:
# CONVERT THE DATASET to regular format(2 Column)
to_be_df = {}
for row in df.iterrows():
    row = row[1]
    image_id = row.image_name
    image_path = f"../input/gwd-test/train/{image_id}.jpg"
    
    if row.PredString == 'no_box':
        to_be_df[image_path] = [np.zeros((0, 4))]
        continue

    
    ALL_BOUNDING_BOX = [x for x in row.PredString.split(";")]
    for bounding_box in ALL_BOUNDING_BOX:
        bounding_box = [int(x) for x in bounding_box.split(" ")]
        x1, y1, x2, y2 = bounding_box
    
        bounding_box = np.array([x1, y1, x2, y2])

        if image_path in to_be_df:
            to_be_df[image_path] += [bounding_box]
        else:
            to_be_df[image_path] = [bounding_box]

for key in to_be_df:
    if np.stack(to_be_df[key]).shape == (1, 0, 4):
        to_be_df[key] = to_be_df[key][0]
        continue
    to_be_df[key] = np.stack(to_be_df[key])
    

    
# Convert to DF
new_df = {'image_name': [], 'bounding_box':  []}
for key in to_be_df:
    new_df['image_name'].append(key)
    new_df['bounding_box'].append(to_be_df[key])

In [ ]:
df = pd.DataFrame(new_df)

In [ ]:
def _bytes_feature(value, is_list=False):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    
    if not is_list:
        value = [value]
    
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def _float_feature(value, is_list=False):
    """Returns a float_list from a float / double."""
        
    if not is_list:
        value = [value]
        
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value, is_list=False):
    """Returns an int64_list from a bool / enum / int / uint."""
        
    if not is_list:
        value = [value]
    
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
def serialize_example(image, bboxes):
    feature = {
        'image': _bytes_feature(tf.io.encode_jpeg(image)),
        'bboxes': _int64_feature(tf.reshape(bboxes, (-1, )), is_list = True)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()
def load_image(filepath):
    loaded = tf.io.read_file(filepath)
    # Load into a PNG
    loaded= tf.io.decode_png(loaded, channels = 3)
    return loaded # Cannot Resize, due to Bounding Box coordinates.
def write_tf_recs(df, filepath):
    # Write a Temporary TFRecord to load in the external data easier.
    with tf.io.TFRecordWriter(filepath) as writer:
        for row in tqdm(df.iterrows()):
            row = row[1]
            image_path = row.image_name
            image = load_image(image_path)
            bounding_boxes = row.bounding_box
            serialized = serialize_example(image, bounding_boxes)
            writer.write(serialized)
    

In [ ]:
write_tf_recs(df, './dataset.tfrec')

# Load 30 Sets of Data(~30 Epochs of Training) and Save under each fold

In [ ]:
RANDOM_Dataset = iter(tf.data.experimental.RandomDataset())
def generate_random_num():
  seed = next(RANDOM_Dataset)
  seed = seed % int(1e5)
  seed = seed / int(1e5)
  return seed
def augment_image(images, bboxes, seeds):
    # image augs from objects
    # Single Augmentation
    #print(images.shape, bboxes.shape)
    images, bboxes = Augments.resize(images, bboxes, seeds)
    
    seed = generate_random_num()
    # Colour and Image Permutation-------------------------
    if seed < 0.5:
      seed = next(RANDOM_Dataset)
      seed1 = next(RANDOM_Dataset)
      images = tf.image.stateless_random_brightness(images, 0.2, seed = (seed, seed1))
    
    
    seed = generate_random_num()
    if seed < 0.5:
      seed = next(RANDOM_Dataset)
      seed1 = next(RANDOM_Dataset)
      images = tf.image.stateless_random_contrast(images, 0.8, 1.2, seed = (seed, seed1))
    
    
    seed = generate_random_num()
    if seed < 0.25:
      seed = next(RANDOM_Dataset)
      seed1 = next(RANDOM_Dataset)
      images = tf.image.stateless_random_hue(images, 0.1, seed = (seed, seed1))
    
    seed = generate_random_num()
    if seed < 0.5:
      seed = next(RANDOM_Dataset)
      seed1 = next(RANDOM_Dataset)
      images = tf.image.stateless_random_saturation(images, 0.8, 1.2, seed = (seed, seed1))

    seed = generate_random_num()
    if seed < 0.01:
      # Grayscale very rarely, but should happen sometimes
      images = tf.image.rgb_to_grayscale(images)
      images = tf.concat([images, images, images], axis = -1)
    
   
    seed = generate_random_num()
    # Noise -----------------
    if seed < 0.5:
      seed = next(RANDOM_Dataset)
      seed1 = next(RANDOM_Dataset)
      gnoise = tf.random.stateless_normal(shape=tf.shape(images), seed = (seed, seed1), mean=0.0, stddev=0.1, dtype=images.dtype)
      images = images + gnoise
    
    seed = generate_random_num()
    if seed < 0.5:
      # Blur the image
      images = tfa.image.gaussian_filter2d(images)
    # Rest of Custom Augments all at once
    
    for augment in Augments.all_augs:
      images, bboxes = augment(images, bboxes, seeds)
    # ----------CLIP the BOUNDING BOXES----------------
    bboxes = tf.reshape(bboxes, (-1, 5))
    bboxes = thresh_by_zero(bboxes)
    bboxes = tf.reshape(bboxes, (-1, 5))

    return images, bboxes
def batched_augment_image(images, bboxes, seeds):  
   
    if Augments.batched_augs != []:
        for aug in Augments.batched_augs:
          images, bboxes = aug(images, bboxes, seeds)
    
    return images, bboxes
def IMAGE_LOAD(example):
    feature_dict = {
        'image': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=''),
        'bboxes': tf.io.VarLenFeature(dtype = tf.int64)
        #'classification': tf.io.VarLenFeature(dtype = tf.int64) 
    }
    features = tf.io.parse_single_example(example, features=feature_dict)
    
    image = features['image']
    bboxes = features['bboxes'] # Classification Exists, but it's just repetition 
    
    # Load image
    image = tf.io.decode_jpeg(image, channels = 3)
    # Cast to Desired Dtype
    image = tf.cast(image, tf.float32) / 255.0
    # Reshape the image 
    image = tf.reshape(image,  DataModule.IMG_SHAPE)
    # Load the Bounding Boxes
    bboxes = tf.cast(tf.sparse.to_dense(bboxes), tf.float32)
    bboxes = tf.reshape(bboxes, (-1, 4)) # (N, 4)
    # Add in the Bounding Box Classes(1)
    N = bboxes.shape[0]
    bboxes = tf.concat([bboxes, tf.ones_like(bboxes)[:, 0:1]], axis = 1)
    return image, bboxes
def get_dfs():
    # Create TFRecordDatasets
    train_dataset = tf.data.TFRecordDataset(['./dataset.tfrec'], num_parallel_reads= AUTO)
    
    
    
    # Options
    options = tf.data.Options()
    options.experimental_deterministic = False
    
    train_dataset = train_dataset.with_options(options)
    # Map Locations
    train_dataset = train_dataset.map(lambda x: IMAGE_LOAD(x), num_parallel_calls = AUTO, deterministic = False)
    
    # Create 4 Seeders(For x1, x2, y1, y2)
    seeder1 = tf.data.experimental.RandomDataset()
    seeder2 = tf.data.experimental.RandomDataset()
    seeder3 = tf.data.experimental.RandomDataset()
    seeder4 = tf.data.experimental.RandomDataset() 

    # Zip Together
    seeders = tf.data.Dataset.zip((seeder1, seeder2, seeder3, seeder4))
    train_dataset = tf.data.Dataset.zip((train_dataset, seeders))
    # Augment the image in the train set, all single_wise transforms:
    train_dataset = train_dataset.map(lambda x, seeds: augment_image(x[0], x[1], seeds), num_parallel_calls = AUTO, deterministic = False)

    # SHUFFLE Datasets
    train_dataset = train_dataset.shuffle(128) 
    
    # batch the Dataset
    # TODO: CREATE CUSTOM BATCHING, ALSO CHECK BOUNDING BOXES BEFORE THIS POINT
    BATCH_SIZE = 16 # need a batch to get mosaic properly.
    train_dataset = train_dataset.padded_batch(BATCH_SIZE, padded_shapes = ((Augments.IMAGE_SIZE, Augments.IMAGE_SIZE, 3), (Augments.MAX_BBOX, 5)), drop_remainder = True)
    
    seeder1 = tf.data.experimental.RandomDataset()
    seeder2 = tf.data.experimental.RandomDataset()
    seeder3 = tf.data.experimental.RandomDataset()
    seeder4 = tf.data.experimental.RandomDataset() 

    # Zip Together
    seeders = tf.data.Dataset.zip((seeder1, seeder2, seeder3, seeder4))
    train_dataset = tf.data.Dataset.zip((train_dataset, seeders))
    
    # Batched Augments
    # ENSURE PROPER BATCHING.
    train_dataset = train_dataset.map(lambda x, seeds: batched_augment_image(x[0], x[1], seeds), num_parallel_calls = AUTO, deterministic = False)

    return train_dataset


In [ ]:
datasets = get_dfs()

In [ ]:
for ex in datasets.take(1):
    break

In [ ]:
def display_bboxes(images, bboxes, idx):
    bboxes = bboxes[idx][bboxes[idx, :, -1] == 1.0].numpy()
    image = images.numpy()[idx]
    for box in bboxes:
        if box[-1] == 0:
          continue
        cv2.rectangle(image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), 220, 3)
    plt.imshow(image)
    plt.show()


Data Saving Pipeline
- Run through the Dataset, saving every example

In [ ]:
def scale_down(image):
    # MinMax Norm
    # Image: Tensor(H, W, 3)
    max_value = tf.reduce_max(image)
    min_value = tf.reduce_min(image)
    
    image = (image - min_value) / (max_value - min_value)
    image = image * 255.0
    image = tf.cast(image, dtype = tf.uint8)
    return image

In [ ]:
NUM_EPOCHS = 10 # max Number
START_EPOCH = 0

In [ ]:
def serialize_example(images, bboxes):
    feature = {
        'image': _bytes_feature(tf.io.encode_jpeg(images)),
        'bboxes': _int64_feature(tf.reshape(bboxes, (-1, )), is_list = True),
        #'classification': _int64_feature(bboxes[:, -1], is_list = True)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()
def write_tf_dataset(num_epochs, start_epoch, fold_idx, is_test = False):
    # Load in the dataset (1 Fold worth)
    dataset = get_dfs()
    for epoch_idx in range(num_epochs):
        epoch_idx = epoch_idx + start_epoch 
        file_path = f"{epoch_idx}_fold_{fold_idx}.tfrec"
        with tf.io.TFRecordWriter(file_path) as writer:
            count = 0
            import time
            cur_time = time.time()
            for images, bboxes in dataset:
                B = images.shape[0]
                for b in range(B):
                    img = scale_down(images[b])
                    bbox = bboxes[b]
                    try:
                        ex = serialize_example(img, bbox)
                        writer.write(ex)
                        count += 1
                        #print(count)
                        if count % 10 == 0:
                            print(count)
                    except:
                        pass
            

In [ ]:
FOLD_NUM = 5

In [ ]:
write_tf_dataset(NUM_EPOCHS, START_EPOCH, FOLD_NUM, is_test = False)